# Tokenize

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
import scipy.stats as stats
from nltk import ngrams
nltk.download('punkt')
stop_words = set(stopwords.words('english'))
from tqdm import tqdm
import plotly
import plotly.express as px

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def count_tokens(abstract, nrange=1):
    stop_words = set(stopwords.words('english'))

    word_tokens = word_tokenize(abstract)
    word_tokens = [w.lower() for w in word_tokens]
    filtered_sentence = [w.lower() for w in word_tokens if w.lower() not in stop_words and w.isalnum() and len(w) > 2]

    all_tokens = []
    size = len(filtered_sentence)
    #print(size)
    for i in range(1, nrange + 1):
      #  print(size)
        if size == 1:
          #  print("in here 1")
            all_tokens.extend(filtered_sentence)
            #print(all_tokens)
        else:
            i=2
            ngrams_list = list(ngrams(filtered_sentence, i))
            if size == 2:
             #   print("in here 2")
                ngrams_list = [ngram for ngram in ngrams_list if len(set(ngram)) == i]  # Include all bigrams for i == 2
            elif size > 2:
              #  print("in here >2")
                ngrams_list = [ngram for ngram in ngrams_list if len(set(ngram)) == i]  # Only include i-tokens for i > 2
            all_tokens.extend(ngrams_list)
            #print(all_tokens)
    count_tok = Counter(all_tokens)
    return dict(count_tok)

In [ ]:

dictionary = {}
for abstract in tqdm(all_hpo):
    all_tokens_counts = count_tokens(abstract, 2)
    dictionary[abstract] = all_tokens_counts

# Get all unique words
all_tokens_counts = [count_tokens(abstract, 2) for abstract in tqdm(all_hpo)]
all_tokens = set()
for count in all_tokens_counts:
    all_tokens.update(count.keys())



100%|██████████| 34/34 [00:00<00:00, 897.27it/s]


In [ ]:
# count frequency of tokens in all HPO
all_counts = [count_tokens(abstract,1) for abstract in tqdm(all_hpo)]

# Get all unique words
all_words = set()
for count in all_counts:
    all_words.update(count.keys())


100%|██████████| 34/34 [00:00<00:00, 1749.30it/s]


In [ ]:
# Create word count matrix
import numpy as np

# Create matrix
word_count_matrix = np.zeros((len(all_tokens_counts), len(all_words)))

# Fill matrix
for i, count in tqdm(enumerate(all_counts)):

    for j, word in enumerate(all_words):

        word_count_matrix[i, j] += count.get(word, 0)

34it [00:00, 18675.53it/s]


In [ ]:
tfIdfTransformer = TfidfTransformer(use_idf=True)
newTfIdf = tfIdfTransformer.fit_transform(word_count_matrix)


<34x52 sparse matrix of type '<class 'numpy.float64'>'
	with 52 stored elements in Compressed Sparse Row format>

In [ ]:
# The more common the word across documents, the lower its score

tf_values = np.squeeze([newTfIdf[:].T.todense().mean(axis=1), newTfIdf[:].T.todense().max(axis=1)]).T
df_tfidf = pd.DataFrame(data = tf_values, columns=["TF-IDF Mean", "TF-IDF Max"], index=list(all_words))
df_tfidf = pd.DataFrame(df_tfidf.sort_values('TF-IDF Max', ascending=False))


,TF-IDF Mean,TF-IDF Max
chorea,0.029412,1.0
"(pes, cavus)",0.029412,1.0
"(intention, tremor)",0.029412,1.0
"(incomprehensible, speech)",0.029412,1.0
"(babinski, sign)",0.029412,1.0


In [ ]:
df_tfidf['TF-IDF Max'].describe()

relevant_hpo = df_tfidf[df_tfidf['TF-IDF Max']>0.1].index.to_list()



,TF-IDF Mean,TF-IDF Max
chorea,0.029412,1.000000
"(pes, cavus)",0.029412,1.000000
"(intention, tremor)",0.029412,1.000000
"(incomprehensible, speech)",0.029412,1.000000
"(babinski, sign)",0.029412,1.000000
scoliosis,0.029412,1.000000
"(hearing, impairment)",0.029412,1.000000
"(limb, ataxia)",0.029412,1.000000
falls,0.029412,1.000000
dysmetria,0.029412,1.000000


# Create dictionary of HPO with CT matches


In [ ]:
matches = {}
all_matches = {}
for token in relevant_hpo:
            highest_match = 90  # Initialize the highest match score
            best_endpoints = []  # Initialize a list to store all the matching sentences

            for endpoint in all_primary_ct:
                match_score = fuzz.token_set_ratio(token, endpoint)  # Calculate the match score
                if match_score > highest_match:
                    highest_match = match_score
                    best_endpoints = [endpoint]
                elif match_score == highest_match:
                    best_endpoints.append(endpoint)

            if best_endpoints:
                found_terms = []  # List to store terms associated with the token

                for term, tokens_list in dictionary.items():
                    if token in tokens_list:  # Check if the token is in the list of associated tokens
                        found_terms.append((token, term))

                if found_terms:
                    for token, term in found_terms:
                        print(f"The term associated with the token '{token}' is '{term}'")
                        # Associate the matched sentences with the term, including the count
                        matches.setdefault(term, []).append({'Sentences': best_endpoints, 'Match Count': len(best_endpoints)})

                else:
                    a = 0

df = pd.DataFrame([(term, match['Sentences'], match['Match Count']) for term, matches_list in matches.items() for match in matches_list], columns=['Term', 'Matched Sentences', 'Match Count'])


The term associated with the token 'falls' is ' Falls'
